<a href="https://colab.research.google.com/github/Ekanshiitk/T20-World-Cup-Score-Predictor/blob/main/CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the required packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt


In [2]:
from tqdm import tqdm
tqdm.pandas()

In [3]:
pd.set_option('display.max_columns',0)
pd.set_option('display.max_colwidth',0)

In [13]:
match_lvl_data = pd.read_csv('/content/664389efa0868_match_level_scorecard.csv')
batsman_lvl_data = pd.read_csv('/content/663e2b548c98c_batsman_level_scorecard.csv')
bowler_lvl_data = pd.read_csv('/content/663e2b2c60743_bowler_level_scorecard.csv')
train_data = pd.read_csv('/content/663e2b6d54457_train_data_with_samplefeatures.csv')
test_data = pd.read_csv('/content/6644a1e287df6_test_data_with_samplefeatures.csv')

In [14]:
match_lvl_data.shape
match_lvl_data.head(15)

,match id,team1,team2,winner,by,win amount,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,umpire1,umpire2,inning1_runs,inning1_wickets,inning1_balls,inning2_runs,inning2_wickets,inning2_balls,team1_id,team1_roster_ids,team2_id,team2_roster_ids,series_type,winner_id,player_of_the_match_id
0,8638034,Nn Ds,Wn,Wn,wickets,9.0,Wn,field,By Ol,Mount Maunganui,2021-01-01,night match,Sr Sh,2020/21,17681,Dn Mw,TJ Pe,148,6,130,152.0,1.0,97.0,17982,7907451.0:4381761.0:31464.0:258649.0:4949790.0:3834305.0:3776849.0:6718340.0:3566240.0:4950364.0:7543647.0,18570,2653993.0:6718326.0:6718382.0:2486896.0:2288789.0:4950224.0:8187878.0:3083667.0:4950294.0:6718396.0:1585464.0,other_domestic,18570,NaN
1,8588005,Me Rs,Sy Tr,Sy Tr,runs,7.0,Sy Tr,field,Ca Ol,Carrara,2021-01-01,night match,Bg Bh Le,2020/21,6663,D Tr,PJ Ge,166,6,106,117.0,2.0,74.0,33942,37351.0:46794.0:5406540.0:2231928.0:181404.0:1594319.0:7534687.0:7537067.0:1905847.0:8339701.0:7620346.0,33963,1506098.0:1749075.0:36665.0:2083409.0:7534652.0:5788320.0:1612610.0:5509524.0:5497274.0:3876613.0:8193310.0,other_domestic,33963,1749075.0
2,8587837,Sy Ss,Be Ht,Be Ht,wickets,4.0,Sy Ss,bat,Be Ct Gd,Brisbane,2021-01-02,night match,Bg Bh Le,2020/21,1476,GJ Dn,PJ Ge,165,8,126,171.0,6.0,119.0,33956,7869987.0:7620283.0:2076192.0:4002340.0:3063696.0:34061.0:3200756.0:4756982.0:7455818.0:49496.0:1506077.0,33921,7620269.0:2286437.0:87191.0:5786766.0:3114803.0:2535420.0:4967738.0:3995991.0:3890984.0:7353828.0:6818776.0:5419546.0,other_domestic,33921,3890984.0
3,8638041,Nn Ds,Oo,Oo,wickets,2.0,Oo,field,By Ol,Mount Maunganui,2021-01-02,night match,Sr Sh,2020/21,17681,GA St,TJ Pe,153,7,128,156.0,8.0,126.0,17982,7907451.0:4381761.0:31464.0:4949790.0:258649.0:3834305.0:3776849.0:7543647.0:6718340.0:3566240.0:7907458.0,18360,2319638.0:256080.0:7918280.0:3913447.0:2690498.0:6129276.0:2666705.0:6317142.0:3834375.0:2236086.0:4195827.0,other_domestic,18360,NaN
4,8587921,Ht Hs,Me Ss,Ht Hs,runs,21.0,Me Ss,field,Be Ol,Hobart,2021-01-02,day/night match,Bg Bh Le,2020/21,6348,GA Ad,NR Je,164,5,126,143.0,9.0,123.0,33928,4223883.0:2161599.0:1655436.0:5788418.0:319948.0:6249256.0:2340372.0:309056.0:3125849.0:6732004.0:6722540.0,33949,363047.0:2275097.0:3901078.0:2275195.0:4230127.0:2720759.0:2337117.0:6308098.0:7877232.0:3731307.0:4985546.0:2336473.0,other_domestic,33928,3125849.0
5,8638048,Ad,Cy,Cy,wickets,10.0,Cy,field,En Pk Or Ol,Auckland,2021-01-03,day match,Sr Sh,2020/21,5081,A Ma,JK By,52,4,33,55.0,0.0,28.0,17583,5764576.0:3069954.0:3935854.0:3913440.0:4949776.0:8144184.0:270262.0:6718438.0:5560190.0:7878765.0:7565697.0,17653,4381747.0:2659243.0:3682160.0:4381705.0:3834361.0:8683562.0:3913433.0:1495731.0:4136404.0:3834319.0:7871310.0,other_domestic,17653,NaN
6,8587907,Ae Ss,Sy Ss,Sy Ss,wickets,7.0,Sy Ss,field,Ca Ol,Carrara,2021-01-03,night match,Bg Bh Le,2020/21,6663,GJ Dn,AK Ws,150,4,125,151.0,3.0,108.0,33914,4685568.0:5056134.0:2285051.0:1635773.0:5469008.0:5776490.0:5554254.0:3901099.0:2170762.0:6718844.0:55299.0,33956,7869987.0:7620283.0:2076192.0:4002340.0:3063696.0:34061.0:3200756.0:4756982.0:7455818.0:2904943.0:7537060.0,other_domestic,33956,7869987.0
7,8587984,Ph Ss,Me Rs,Ph Ss,runs,96.0,Me Rs,field,Ph Sm,Perth,2021-01-03,day match,Bg Bh Le,2020/21,23841,MW Gh,SA Ly,185,3,124,89.0,10.0,80.0,33935,2089079.0:2827327.0:4635658.0:1626526.0:1907163.0:3501889.0:7869994.0:5419574.0:3216569.0:1907352.0:1684276.0,33942,5406540.0:46794.0:37351.0:2669316.0:181404.0:2231928.0:6842170.0:1905847.0:1906708.0:8339701.0:8277716.0,other_domestic,33935,4635658.0
8,8638055,Cl Ds,Nn Ds,Nn Ds,wickets,6.0,Nn Ds,field,Sn Pk,Hamilton,2021-01-04,night match,Sr Sh,2020/21,3541,Gn Wn,JM Dy,117,9,124,122.0,4.0,107.0,17744,1945390.0:6487032.0:3083632.0:6496118.0:2537800.0:1626540.0:3558316.0:4170494.0:8354821.0:5984376.0:8447396.0,17982,7907451.0:4381761.0:31464.0:4949790.0:258649.0:3834305.0:3776849.0:6718340.0:3566240.0:7543647.0:7907458.0,other_domestic,17982,NaN
9,8588166,Me Ss,Ht Hs,Me Ss,runs,10.0

In [15]:
batsman_lvl_data.shape
batsman_lvl_data.head(2)

,match id,batsman,batsman_id,batsman_details,is_batsman_captain,is_batsman_keeper,inning,runs,balls_faced,over_faced_first,wicket kind,out_by_bowler,out_by_fielder,bowler_id,bowler_details,is_bowler_keeper,is_bowler_captain,strike_rate,Fours,Sixes,match_dt
0,8638034,KD Ce,7907451.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,7,5,1.1,caught,JS Nm,TM Jn,2486896.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,140.0,1.0,NaN,2021-01-01
1,8638034,TL St,4381761.0,NZ:Right-hand bat:None:,0.0,1.0,1,46,46,1.2,caught,R Ra,MG Bl,6718382.0,NZ:Left-hand bat:Slow left-arm orthodox:,0.0,0.0,100.0,4.0,1.0,2021-01-01


In [16]:
bowler_lvl_data.shape
bowler_lvl_data.head(2)

,match id,bowler,bowler_id,bowler_details,is_bowler_captain,is_bowler_keeper,inning,runs,wicket_count,balls_bowled,economy,maiden,dots,Fours,Sixes,wides,noballs,match_dt
0,8638034,BV Ss,6718396.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,21,0,12,10.5,0,3.0,1,1,3,1,2021-01-01
1,8638034,HK Bt,1585464.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,1,34,1,24,8.5,0,7.0,3,1,2,0,2021-01-01


In [17]:
train_data.shape
train_data.head(2)

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15
0,9331181,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:3299407.0:232000.0:7883504.0:8725723.0:7878989.0:2526390.0:8725730.0:4017523.0,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:3698337.0:5058192.0:6844718.0:4566526.0:6596582.0:4161975.0:7952804.0:8465057.0,Hl Ph,12634,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.0,100.0,157.178571
1,8797060,Ed,20,2089079.0:6139370.0:2076192.0:62432.0:2083409.0:172199.0:2022957.0:1482998.0:1711492.0:4507208.0:3231437.0,Wt Is,41,4690258.0:4069666.0:4230127.0:1942317.0:1613926.0:2740408.0:4739552.0:5744850.0:4690104.0:3715697.0:3468870.0,Ed,20,Wt Is,field,Kn Ol Bn Bs,Bridgetown,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.0,50.0,103.500000


In [18]:
test_data.shape
test_data.head(15)


,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15
0,9250275,Jy,28594,7438955.0:8271969.0:8369661.0:3685247.0:2590251.0:7785497.0:7785553.0:3296264.0:4613622.0:2904796.0:3685289.0,Ud Ss of Aa,90,2436944.0:8117500.0:6703528.0:3010748.0:1613870.0:8934764.0:2721480.0:1632819.0:2789604.0:6489846.0:1565507.0,Ud Ss of Aa,field,Bo Ac Cb,Bulawayo,2022-07-11,day match,Ud Ss of Aa tr of Ze,2022,3226,0.000000,0.019608,NaN,0.00,NaN
1,9262189,Ga An Ws,36084,3715690.0:6818622.0:4069666.0:393014.0:4690188.0:4739552.0:4690328.0:5744850.0:4739580.0:9266102.0:284339.0,Ja Ts,36098,4690258.0:3761246.0:8464385.0:5742470.0:3150720.0:4690104.0:181404.0:1594319.0:5509524.0:4239794.0:2036649.0,Ja Ts,field,Pe Sm Ga,Providence,2022-09-21,night match,Cn Pr Le,2022,13915,0.615385,0.344262,151.285714,66.67,153.500000
2,9128776,Rn Rs,30428,8058959.0:2162782.0:2981614.0:7833195.0:7556051.0:2486896.0:184960.0:1945397.0:6420126.0:3011735.0:6347494.0,Lw Sr Gs,48334,2654014.0:2954769.0:8058903.0:3479860.0:3299407.0:2275097.0:2740408.0:3865077.0:7924048.0:4859490.0:8228100.0,Rn Rs,bat,Be Sm Mi,Mumbai,2022-05-15,night match,In Pr Le,2022,2764,0.842105,0.753086,171.066667,100.00,179.625000
3,9586919,Ja Ts,36098,4690258.0:8464385.0:1613898.0:5744780.0:3150720.0:4690104.0:1594319.0:5509524.0:2036649.0:3023383.0:5471948.0,St La Ks,36112,2331475.0:313809.0:391103.0:2742837.0:2097017.0:7921535.0:7459647.0:8464343.0:4690230.0:8339701.0:4702774.0,St La Ks,field,Dn Sy Nl Ct Sm Gs It St La,Gros Islet,2023-08-16,night match,Cn Pr Le,2023,11892,0.285714,1.487805,166.000000,75.00,155.500000
4,9128538,Ci Sr Ks,30414,7422673.0:249087.0:62432.0:232000.0:5001170.0:1642738.0:196580.0:360086.0:2022957.0:7968225.0:7879829.0,Gt Ts,48341,1958683.0:7491224.0:3339160.0:5452754.0:2252452.0:2966879.0:5554254.0:4690230.0:3456424.0:3373285.0:8118053.0,Gt Ts,field,Ma Ct An Sm Pe,Pune,2022-04-17,night match,In Pr Le,2022,18752,2.375000,0.310330,169.933333,0.00,164.125000
5,9516653,Pb Ks,30407,8127181.0:197658.0:4239038.0:2827327.0:5053082.0:4640824.0:5038046.0:8180500.0:2035102.0:7453697.0:7881845.0:5788418.0,Mi Is,30435,238727.0:5043310.0:7537004.0:3125562.0:6249256.0:8191868.0:8058924.0:230775.0:4688998.0:8118914.0:8713270.0:8442286.0,Mi Is,field,Pb Ct An IS Ba Sm Mi Ch,Chandigarh,2023-05-03,night match,In Pr Le,2023,7118,0.400000,0.672131,177.066667,50.00,184.500000
6,9516534,Ss Hd,36014,6381962.0:7491294.0:3127354.0:4203499.0:3057312.0:2789079.0:4874820.0:5038018.0:2282125.0:7570107.0:8725709.0:6596582.0,Ci Sr Ks,30414,7422673.0:2653993.0:1945425.0:232000.0:62432.0:5001170.0:1642738.0:196580.0:5757884.0:7968225.0:8363578.0:8228219.0,Ci Sr Ks,field,MA Cm Sm Ck Ci,Chennai,2023-04-21,night match,In Pr Le,2023,2050,0.681818,0.344262,183.466667,25.00,156.350000
7,9085474,St,9701,7694581.0:6139370.0:2231928.0:3239102.0:2535420.0:6320362.0:5288226.0:334963.0:3252395.0:2850868.0:55299.0,De,6698,7295028.0:2433640.0:325814.0:4170186.0:3901078.0:6718788.0:2872225.0:3991700.0:4613580.0:8056705.0:6005866.0,De,field,Te Cr As Cy Gd Tn,Taunton,2022-07-09,night match,Vy Bt,2022,1042,3.200000,0.603960,163.066667,0.00,177.321429
8,9433752,He,7727,2076192.0:4223883.0:8271913.0:5241564.0:1482998.0:2020332.0:3162487.0:1482249.0:2173688.0:5788418.0:8833075.0,Gn,7573,5164844.0:319948.0:2707431.0:1722048.0:7734271.0:5161652.0:8110185.0:5192830.0:7664957.0:3212292.0:8339701.0,Gn,field,Te Re Bl Sn,Southampton,2023-06-30,day/night match,Vy Bt,2023,8301,2.428571,2.904762,167.000000,100.00,151.600000
9,9516359,Rn Rs,30428,8058959.0:2162782.0:2981614.0:7833195.0:7556051.0:4690188.0:184960.0:2740408.0:1945397.0:7581223.0:3011735.0:4901182.0,Ss Hd,36014,7491294.0:2789079.0:3127354.0:6381962.0:5038018.0:5764576.0:8228408.0:1711492.0:2282125.0:8725709.0:5618038.0:6819238.0,Ss Hd,field,Rv Gi Il Sm Ul Hd,Hyde

In [19]:
match_lvl_data = match_lvl_data.fillna(method='ffill')
batsman_lvl_data = batsman_lvl_data.fillna(method='ffill')
bowler_lvl_data = bowler_lvl_data.fillna(method='ffill')


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Merge datasets
data = match_lvl_data.merge(batsman_lvl_data, on='match id', how='left')
data = data.merge(bowler_lvl_data, on='match id', how='left')
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Iterate through the categorical columns and encode them
for col in ['winner', 'by', 'toss winner', 'toss decision', 'city', 'match_dt_x', 'lighting', 'series_name', 'season', 'umpire1', 'umpire2', 'team1_roster_ids', 'team2_roster_ids', 'series_type', 'batsman', 'batsman_details', 'wicket kind', 'out_by_bowler', 'out_by_fielder', 'bowler_details_x', 'match_dt_y', 'bowler', 'bowler_details_y', 'match_dt']:
    data[col] = label_encoder.fit_transform(data[col])

# Alternatively, you can use the `apply()` method:
# data = data.apply(lambda x: label_encoder.fit_transform(x) if x.dtype == 'object' else x)


# Environmental and contextual factors
data['day_night_match'] = data['lighting'].apply(lambda x: 1 if x == 'night match' else 0)
data['home_crowd_support'] = data['series_type'].apply(lambda x: 1 if x=='domestic' else 0)

# Encode categorical variables
label_encoder = LabelEncoder()
data['team1'] = label_encoder.fit_transform(data['team1'])
data['team2'] = label_encoder.fit_transform(data['team2'])
data['venue'] = label_encoder.fit_transform(data['venue'])

# Define features and target variable
X = data.drop(['winner_id', 'match id'], axis=1)
y = data['winner_id']

# Encode the target variable
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost model
from xgboost import XGBClassifier
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')